In [17]:
path="CBOW(Ass5)/CBOW.txt"

In [18]:
with open(path,'r') as file:
    file_contents=file.read()

In [19]:
file_contents

'The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \n\nFurther, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission â€“transmission of the virus before the appearance of symptoms â€“ is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \n\nThe reproductive number â€“ the number of secondary infections generated from one infected individual â€“ is understood to 

In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
sentences = file_contents.split('.')

# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1

# Generate context-target pairs for training
window_size = 3
tokenized_sentences = tokenizer.texts_to_sequences(sentences)

data, labels = [], []
for sentence in tokenized_sentences:
    for i, target_word in enumerate(sentence):
        context = [
            sentence[j] for j in range(i - window_size, i + window_size + 1)
            if j != i and 0 <= j < len(sentence)
        ]
        data.append(context)
        labels.append(target_word)

# Convert data and labels to numpy arrays
data = pad_sequences(data)
labels = np.array(labels)

In [25]:
model=Sequential()
model.add(Embedding(input_dim=total_words,output_dim=50,input_length=window_size*2))
model.add(GlobalAveragePooling1D())
model.add(Dense(total_words,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 6, 50)             5150      
                                                                 
 global_average_pooling1d_2  (None, 50)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 103)               5253      
                                                                 
Total params: 10403 (40.64 KB)
Trainable params: 10403 (40.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(data,labels,epochs=200,verbose=1)

Epoch 1/200
7/7 [==============================] - 0s 4ms/step - loss: 4.6345 - accuracy: 0.0101
Epoch 2/200
7/7 [==============================] - 0s 806us/step - loss: 4.6249 - accuracy: 0.0556
Epoch 3/200
7/7 [==============================] - 0s 1ms/step - loss: 4.6172 - accuracy: 0.1061
Epoch 4/200
7/7 [==============================] - 0s 1ms/step - loss: 4.6098 - accuracy: 0.1465
Epoch 5/200
7/7 [==============================] - 0s 1ms/step - loss: 4.6021 - accuracy: 0.1919
Epoch 6/200
7/7 [==============================] - 0s 1ms/step - loss: 4.5942 - accuracy: 0.1818
Epoch 7/200
7/7 [==============================] - 0s 965us/step - loss: 4.5857 - accuracy: 0.1919
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 4.5770 - accuracy: 0.2020
Epoch 9/200
7/7 [==============================] - 0s 751us/step - loss: 4.5676 - accuracy: 0.1919
Epoch 10/200
7/7 [==============================] - 0s 1ms/step - loss: 4.5576 - accuracy: 0.1869
Epoch 11/200
7/7 [=====

In [27]:
word_embeddings= model.layers[0].get_weights()[0]

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

target_word = 'influenza'
target_embedding = word_embeddings[tokenizer.word_index[target_word]]

similarities = cosine_similarity(target_embedding.reshape(1, -1), word_embeddings)[0]
most_similar_indices = similarities.argsort()[-5:][::-1]
    
most_similar_words = [word for word, idx in tokenizer.word_index.items() if idx in most_similar_indices]

print(f"Most similar words to '{target_word}': {most_similar_words}")

Most similar words to 'influenza': ['influenza', 'can', 'means', 'spread', 'people']
